In [1]:
#!pip install tensorflow-gpu==2.8.3
#!pip install tensorflow==2.8.3

In [2]:
import gc

gc.enable()
gc.collect()

219

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
PATH = ''

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'valid')

BATCH_SIZE = 1024
IMG_SIZE = (224, 224) 

In [6]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    label_mode='categorical')

Found 15867 files belonging to 4 classes.


In [7]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    label_mode='categorical')

Found 3866 files belonging to 4 classes.


In [8]:
class_names = train_dataset.class_names

In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

In [10]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(
        'horizontal_and_vertical'),  # 'horizontal_and_vertical'
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomTranslation(0.2, 0.2),
    tf.keras.layers.RandomZoom(0.2, 0.2),
    tf.keras.layers.RandomContrast(0.4),
])

In [11]:
# preprocess_input = tf.keras.applications.efficientnet_v2.preprocess_input  # EfficientnetV2!
preprocess_input = tf.keras.applications.mobilenet_v3.preprocess_input

In [12]:
IMG_SHAPE = IMG_SIZE + (3, )

# # EfficientnetV2!
# base_model = tf.keras.applications.EfficientNetV2S(input_shape=IMG_SHAPE,
#                                                include_top=False,
#                                                weights='imagenet')

base_model = tf.keras.applications.MobileNetV3Large(input_shape=IMG_SHAPE,
                                                    include_top=False,
                                                    weights='imagenet')

In [13]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)

In [14]:
base_model.trainable = False

In [15]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)

In [16]:
prediction_layer = tf.keras.layers.Dense(4, activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(1024, 4)


In [17]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [18]:
print("Number of layers in the base model: ", len(base_model.layers))
N = len(base_model.layers)
N = int(N * 0.8)
N

Number of layers in the base model:  263


210

In [19]:
base_model.trainable = True

fine_tune_at = N
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [20]:
base_learning_rate = 0.000001
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [21]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 MobilenetV3large (Functiona  (None, 7, 7, 960)        2996352   
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 960)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 960)               0         
                                                                 
 dense (Dense)               (None, 4)                 3844  

In [22]:
checkpoint_path = "save2/ckpt-loss={loss:.4f}"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, verbose=1, save_weights_only=False)

In [23]:
class_names

['Cygnus_bewickii', 'Cygnus_cygnus', 'Cygnus_olor', 'other_bird']

In [24]:
from collections import defaultdict

ACC = []

class AccuracyCallback(tf.keras.callbacks.Callback):

    def __init__(self, validation_dataset):
        self.validation_dataset = validation_dataset

    def on_epoch_end(self, epoch, logs=None):

        acc = defaultdict(lambda: [])

        num_valid_batches = tf.data.experimental.cardinality(self.validation_dataset)

        for images, labels in self.validation_dataset.take(num_valid_batches):

            pred = self.model.predict(images, verbose=0).argmax(axis=1)
            true = labels.numpy().argmax(axis=1)

            for t, p in zip(true, pred):
                acc[t].append(int(t == p))

        message = '\n'
        for k in class_names:
            v = acc[class_names.index(k)]
            message += f"acc - {k} : {np.round(np.mean(v), 3)}%, (n={len(v)})\n"
        
        ACC.append(message)

        print(message + '\n')

In [ ]:
history = model.fit(train_dataset,
                    epochs=50,
                    validation_data=validation_dataset, callbacks =  [checkpoint_callback, 
                                                                      AccuracyCallback(validation_dataset)])

Epoch 1/50
16/16 [==============================] - ETA: 0s - loss: 2.2823 - categorical_accuracy: 0.1941
Epoch 1: saving model to save2/ckpt-loss=2.2823


2023-05-20 20:50:05.062183: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: save2/ckpt-loss=2.2823/assets

acc - Cygnus_bewickii : 0.007%, (n=1313)
acc - Cygnus_cygnus : 0.014%, (n=1289)
acc - Cygnus_olor : 0.972%, (n=887)
acc - other_bird : 0.019%, (n=377)


16/16 [==============================] - 86s 5s/step - loss: 2.2823 - categorical_accuracy: 0.1941 - val_loss: 2.3498 - val_categorical_accuracy: 0.2318
Epoch 2/50
16/16 [==============================] - ETA: 0s - loss: 2.2488 - categorical_accuracy: 0.2083
Epoch 2: saving model to save2/ckpt-loss=2.2488
INFO:tensorflow:Assets written to: save2/ckpt-loss=2.2488/assets

acc - Cygnus_bewickii : 0.008%, (n=1313)
acc - Cygnus_cygnus : 0.016%, (n=1289)
acc - Cygnus_olor : 0.972%, (n=887)
acc - other_bird : 0.019%, (n=377)


16/16 [==============================] - 72s 4s/step - loss: 2.2488 - categorical_accuracy: 0.2083 - val_loss: 2.3045 - val_categorical_accuracy: 0.2328
Epoch 3/50
16/16 [==============================] - ETA: 0s - loss: 2.2014 - categorical_accuracy: 0.2

Epoch 17/50
16/16 [==============================] - ETA: 0s - loss: 1.6407 - categorical_accuracy: 0.2981
Epoch 30: saving model to save2/ckpt-loss=1.6407
INFO:tensorflow:Assets written to: save2/ckpt-loss=1.6407/assets

acc - Cygnus_bewickii : 0.114%, (n=1313)
acc - Cygnus_cygnus : 0.099%, (n=1289)
acc - Cygnus_olor : 0.891%, (n=887)
acc - other_bird : 0.027%, (n=377)


16/16 [==============================] - 72s 4s/step - loss: 1.6407 - categorical_accuracy: 0.2981 - val_loss: 1.6436 - val_categorical_accuracy: 0.2788
Epoch 31/50
16/16 [==============================] - ETA: 0s - loss: 1.6355 - categorical_accuracy: 0.2933
Epoch 31: saving model to save2/ckpt-loss=1.6355
INFO:tensorflow:Assets written to: save2/ckpt-loss=1.6355/assets

acc - Cygnus_bewickii : 0.118%, (n=1313)
acc - Cygnus_cygnus : 0.104%, (n=1289)
acc - Cygnus_olor : 0.888%, (n=887)
acc - other_bird : 0.027%, (n=377)


16/16 [==============================] - 68s 4s/step - loss: 1.6355 - categorical_accuracy: 0.293

In [ ]:
model.save('model_mobilenet_stage2.h5') # needs save to make sure it will load, if checkpoint is corrupted

In [ ]:
with open('acc_history.txt', 'w') as f:
    f.write('\n\n'.join(ACC))